# Dice (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common

## pow6

In [ ]:
let pow6 = 1 |> Seq.unfold (fun state -> Some (state, state * 6)) |> Seq.cache

In [ ]:
//// test

pow6
|> Seq.take 8
|> Seq.toList
|> _equal [ 1; 6; 36; 216; 1296; 7776; 46656; 279936 ]

[ 1, 6, 36, 216, 1296, 7776, 46656, 279936 ]


## rollAcc

In [ ]:
let rec rollAcc log rolls power acc =
    match rolls with
    | _ when power < 0 ->
        if log then printfn $"rollAcc / power: {power} / acc: {acc}"
        Some (acc + 1, rolls)
    | [] -> None
    | roll :: rest when roll > 1 ->
        let coeff = pow6 |> Seq.item power
        let value = (roll - 1) * coeff
        if log then printfn $"rollAcc / power: {power} / acc: {acc} / roll: {roll} / value: {value}"
        rollAcc log rest (power - 1) (acc + value)
    | roll :: rest ->
        if log then printfn $"rollAcc / power: {power} / acc: {acc} / roll: {roll}"
        rollAcc log rest (power - 1) acc

In [ ]:
//// test

rollAcc true [6; 5; 4; 3; 2] 0 1000
|> _equal (Some (1006, [5; 4; 3; 2]))

rollAcc / power: 0 / acc: 1000 / roll: 6 / value: 5
rollAcc / power: -1 / acc: 1005
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1006
      - [ 5, 4, 3, 2 ]


In [ ]:
//// test

rollAcc true [6; 5; 4; 3; 2] 1 1000
|> _equal (Some (1035, [4; 3; 2]))

rollAcc / power: 1 / acc: 1000 / roll: 6 / value: 30
rollAcc / power: 0 / acc: 1030 / roll: 5 / value: 4
rollAcc / power: -1 / acc: 1034
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1035
      - [ 4, 3, 2 ]


In [ ]:
//// test

rollAcc true [6; 5; 4; 3; 2] 2 1000
|> _equal (Some (1208, [3; 2]))

rollAcc / power: 2 / acc: 1000 / roll: 6 / value: 180
rollAcc / power: 1 / acc: 1180 / roll: 5 / value: 24
rollAcc / power: 0 / acc: 1204 / roll: 4 / value: 3
rollAcc / power: -1 / acc: 1207
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1208
      - [ 3, 2 ]


## fixedRoll

In [ ]:
let fixedRoll log max rolls =
    let rec rollMax power =
        match rollAcc log rolls power 0 with
        | Some (result, _) when result >= 1 && result <= max -> Some result
        | _ -> None

    rollMax (List.length rolls - 1)

In [ ]:
//// test

fixedRoll true 2000 [1; 5; 4; 4; 5]
|> _equal (Some 995)

rollAcc / power: 4 / acc: 0 / roll: 1
rollAcc / power: 3 / acc: 0 / roll: 5 / value: 864
rollAcc / power: 2 / acc: 864 / roll: 4 / value: 108
rollAcc / power: 1 / acc: 972 / roll: 4 / value: 18
rollAcc / power: 0 / acc: 990 / roll: 5 / value: 4
rollAcc / power: -1 / acc: 994
FSharpOption<Int32>
      Value: 995


In [ ]:
//// test

fixedRoll true 2000 [2; 2; 6; 4; 5]
|> _equal (Some 1715)

rollAcc / power: 4 / acc: 0 / roll: 2 / value: 1296
rollAcc / power: 3 / acc: 1296 / roll: 2 / value: 216
rollAcc / power: 2 / acc: 1512 / roll: 6 / value: 180
rollAcc / power: 1 / acc: 1692 / roll: 4 / value: 18
rollAcc / power: 0 / acc: 1710 / roll: 5 / value: 4
rollAcc / power: -1 / acc: 1714
FSharpOption<Int32>
      Value: 1715


In [ ]:
//// test

fixedRoll true 2000 [4; 1; 1; 2; 3]
|> _equal None

rollAcc / power: 4 / acc: 0 / roll: 4 / value: 3888
rollAcc / power: 3 / acc: 3888 / roll: 1
rollAcc / power: 2 / acc: 3888 / roll: 1
rollAcc / power: 1 / acc: 3888 / roll: 2 / value: 6
rollAcc / power: 0 / acc: 3894 / roll: 3 / value: 2
rollAcc / power: -1 / acc: 3896
<null>


## numDices

In [ ]:
let numDices log max =
    let rec numDices' n p =
        if log then printfn $"numDices / max: {max} / n: {n} / p: {p}"
        if p >= max
        then n
        else numDices' (n + 1) (p * 6)
    if max = 1
    then 1
    else numDices' 0 1

In [ ]:
//// test

numDices true 36
|> _equal 2

numDices / max: 36 / n: 0 / p: 1
numDices / max: 36 / n: 1 / p: 6
numDices / max: 36 / n: 2 / p: 36
2


In [ ]:
//// test

numDices true 7777
|> _equal 6

numDices / max: 7777 / n: 0 / p: 1
numDices / max: 7777 / n: 1 / p: 6
numDices / max: 7777 / n: 2 / p: 36
numDices / max: 7777 / n: 3 / p: 216
numDices / max: 7777 / n: 4 / p: 1296
numDices / max: 7777 / n: 5 / p: 7776
numDices / max: 7777 / n: 6 / p: 46656
6


## progressiveRoll

In [ ]:
#if FABLE_COMPILER_RUST
let rollD6 () : int =
#if CHAIN
    Fable.Core.Rust.emitExpr () "1"
#else
    Fable.Core.Rust.emitExpr () "rand::Rng::gen_range(&mut rand::thread_rng(), 1..7)"
#endif
#else
let private random = System.Random ()
let rollD6 () =
    random.Next (1, 7)
#endif

## progressiveRoll

In [ ]:
let progressiveRoll log reroll max =
    let rec rollMax power =
        let rec loop rolls size =
            if size < power + 1
            then loop (rollD6 () :: rolls) (size + 1)
            else
                match rollAcc log rolls power 0 with
                | Some (result, _) when result <= max -> result
                | _ when reroll -> loop (List.init power (fun _ -> rollD6 ())) power
                | _ -> loop (rollD6 () :: rolls) (size + 1)
        loop [] 0
    rollMax ((numDices log max) - 1)

In [ ]:
//// test

progressiveRoll false false 1
|> _equal 1

1


In [ ]:
//// test

[1..100]
|> List.iter (fun n ->
    [0..1]
    |> List.iter (fun reroll ->
        [1..3000]
        |> List.map (fun _ -> progressiveRoll false (reroll = 1) n)
        |> List.groupBy id
        |> List.length
        |> __equal false n
    )
)

## main

In [ ]:

let main args =
    let result = fixedRoll true 2000 [1; 5; 4; 4; 5]
    trace Debug (fun () -> $"main / result: {result |> Option.defaultValue -1}") getLocals
    0